**Plan**

**1. Custom Layers**

**2. Custom Models**

**3. Callbacks and Custom Training Loops**

**4. Model Saving and Loading**

**<h2>1. Custom Layers</h2>**

A custom layer is a subclass of tf.keras.layers.Layer that allows you to define its own forward pass and manage its own weights.

**Key Methods:**

- init: Initialize the layer.
- build: Create the weights of the layer.
- call: Define the forward pass.
- get_config: Serialize the layer configuration.

**Example 1: Simple Custom Layer**

**Objective:** Create a layer that adds a constant value to the input

In [1]:
import tensorflow as tf

class AddConstantLayer(tf.keras.layers.Layer):
    def __init__(self, constant_value=1, **kwargs):
        super(AddConstantLayer, self).__init__(**kwargs)
        self.constant_value = constant_value

    def call(self, inputs):
        return inputs + self.constant_value

# Usage
inputs = tf.keras.Input(shape=(4,))
outputs = AddConstantLayer(constant_value=5)(inputs)
model = tf.keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4)]               0         
                                                                 
 add_constant_layer (AddCon  (None, 4)                 0         
 stantLayer)                                                     
                                                                 
Total params: 0 (0.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


**Example 2: Custom Layer with Trainable Weights**

**Objective:** Create a layer that learns a bias term

In [2]:
class BiasLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(BiasLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.bias = self.add_weight(shape=(input_shape[-1],), initializer='zeros', trainable=True)
        super(BiasLayer, self).build(input_shape)

    def call(self, inputs):
        return inputs + self.bias

# Usage
inputs = tf.keras.Input(shape=(4,))
outputs = BiasLayer()(inputs)
model = tf.keras.Model(inputs, outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 4)]               0         
                                                                 
 bias_layer (BiasLayer)      (None, 4)                 4         
                                                                 
Total params: 4 (16.00 Byte)
Trainable params: 4 (16.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


**Example 3: Custom Dense Layer with Activation**

**Objective:** Create a dense layer with activation

In [3]:
import tensorflow as tf

class CustomDenseLayer(tf.keras.layers.Layer):
    def __init__(self, units=32, activation=None, **kwargs):
        super(CustomDenseLayer, self).__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        self.kernel = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer='glorot_uniform',
            trainable=True
        )
        self.bias = self.add_weight(
            shape=(self.units,),
            initializer='zeros',
            trainable=True
        )
        super(CustomDenseLayer, self).build(input_shape)

    def call(self, inputs):
        output = tf.matmul(inputs, self.kernel) + self.bias
        if self.activation:
            output = self.activation(output)
        return output



# Usage
inputs = tf.keras.Input(shape=(4,))
outputs = CustomDenseLayer(units=8, activation='relu')(inputs)
model = tf.keras.Model(inputs, outputs)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 4)]               0         
                                                                 
 custom_dense_layer (Custom  (None, 8)                 40        
 DenseLayer)                                                     
                                                                 
Total params: 40 (160.00 Byte)
Trainable params: 40 (160.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


**Example 4: Custom Layer with Configuration**

**Objective:** Create a layer that can be serialized and deserialized

In [ ]:
class SerializableLayer(tf.keras.layers.Layer):
    def __init__(self, constant_value=1, **kwargs):
        super(SerializableLayer, self).__init__(**kwargs)
        self.constant_value = constant_value

    def call(self, inputs):
        return inputs + self.constant_value

    def get_config(self):
        config = super(SerializableLayer, self).get_config()
        config.update({'constant_value': self.constant_value})
        return config

# Usage
inputs = tf.keras.Input(shape=(4,))
outputs = SerializableLayer(constant_value=5)(inputs)
model = tf.keras.Model(inputs, outputs)

# Serialize and deserialize
config = model.layers[1].get_config()
new_layer = SerializableLayer.from_config(config)

**<h2>2. Custom Models</h2>**

A custom model is a subclass of tf.keras.Model that allows you to define custom training, evaluation, and prediction logic.

**Key Methods:**

- __init__: Initialize the model.
- call: Define the forward pass.
- train_step: Custom training logic.
- test_step: Custom evaluation logic.
- compile: Compile the model with loss, optimizer, and metrics.
- metrics: Define custom metrics.

**Example 1: Simple Custom Model**

**Objective:** Create a model using functional API

In [ ]:
class SimpleCustomModel(tf.keras.Model):
    def __init__(self, **kwargs):
        super(SimpleCustomModel, self).__init__(**kwargs)
        self.dense = tf.keras.layers.Dense(1)

    def call(self, inputs):
        return self.dense(inputs)

# Usage
model = SimpleCustomModel()
model.compile(optimizer='adam', loss='mse')

# Generate some dummy data
import numpy as np
x_train = np.random.random((100, 4))
y_train = np.random.random((100, 1))

# Train the model
model.fit(x_train, y_train, epochs=5)

**Example 2: Custom Model with Custom Layer**

**Objective:** Use a custom layer in the model

In [ ]:
class ModelWithCustomLayer(tf.keras.Model):
    def __init__(self, **kwargs):
        super(ModelWithCustomLayer, self).__init__(**kwargs)
        self.dense1 = tf.keras.layers.Dense(4, activation='relu')
        self.custom_layer = CustomDenseLayer(units=8, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.custom_layer(x)
        return self.dense2(x)

# Usage
model = ModelWithCustomLayer()
model.compile(optimizer='adam', loss='mse')

# Generate some dummy data
import numpy as np
x_train = np.random.random((100, 4))
y_train = np.random.random((100, 1))

# Train the model
model.fit(x_train, y_train, epochs=5)

**Example 3: Custom Training Step**

**Objective:** Override **train_step** to customize training logic

In [ ]:
class CustomTrainingModel(tf.keras.Model):
    def __init__(self, **kwargs):
        super(CustomTrainingModel, self).__init__(**kwargs)
        self.dense1 = tf.keras.layers.Dense(4, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1)

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)

    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        # calculate the gradients
        gradients = tape.gradient(loss, self.trainable_variables)
        # update weights
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        # update metrics
        self.compiled_metrics.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}

# Usage
import numpy as np
x_train = np.random.random((100, 4))
y_train = np.random.random((100, 1))
model = CustomTrainingModel()
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.fit(x_train, y_train, epochs=5)

**Example 4: Custom Test Step and Metrics**

**Objective:** Override **test_step** and define custom metrics

In [ ]:
class AdvancedCustomModel(tf.keras.Model):
    def __init__(self, **kwargs):
        super(AdvancedCustomModel, self).__init__(**kwargs)
        self.dense1 = tf.keras.layers.Dense(4, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1)
        self.mae_metric = tf.keras.metrics.MeanAbsoluteError(name='mae')

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)

    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        self.compiled_metrics.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        x, y = data
        y_pred = self(x, training=False)
        loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        self.compiled_metrics.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}

# Usage
import numpy as np
x_train = np.random.random((100, 4))
y_train = np.random.random((100, 1))
x_test = np.random.random((20, 4))
y_test = np.random.random((20, 1))
model = AdvancedCustomModel()
model.compile(optimizer='adam', loss='mse', metrics=[model.mae_metric])
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

**Example 3: Advanced Custom Model with Validation Metrics**

**Objective:** Add validation metrics handling in the custom model.

In [ ]:
class AdvancedCustomModel(tf.keras.Model):
    def __init__(self, **kwargs):
        super(AdvancedCustomModel, self).__init__(**kwargs)
        self.dense1 = tf.keras.layers.Dense(4, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1)
        self._mae_metric = tf.keras.metrics.MeanAbsoluteError(name="mae")
        self._mse_metric = tf.keras.metrics.MeanSquaredError(name="mse")

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)

    @property
    def metrics(self):
        return [self._mae_metric, self._mse_metric]

    def compile(self, optimizer, loss, **kwargs):
        super(AdvancedCustomModel, self).compile(**kwargs)
        self.optimizer = optimizer
        self.compiled_loss = loss

    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)
            loss = self.compiled_loss(y, y_pred)
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        self._mae_metric.update_state(y, y_pred)
        self._mse_metric.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        x, y = data
        y_pred = self(x, training=False)
        loss = self.compiled_loss(y, y_pred)
        self._mae_metric.update_state(y, y_pred)
        self._mse_metric.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}

# Usage
model = AdvancedCustomModel()
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError())

# Generate some dummy data
x_train = np.random.random((100, 4))
y_train = np.random.random((100, 1))
x_val = np.random.random((20, 4))
y_val = np.random.random((20, 1))

# Train the model
model.fit(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

# Evaluate the model
model.evaluate(x_val, y_val)

**Example: Training with train_on_batch**

**Objective:** Train the model using train_on_batch for more granular control over the training process.

In [ ]:
import tensorflow as tf
import numpy as np

class AdvancedCustomModel(tf.keras.Model):
    def __init__(self, **kwargs):
        super(AdvancedCustomModel, self).__init__(**kwargs)
        self.dense1 = tf.keras.layers.Dense(4, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1)
        self._mae_metric = tf.keras.metrics.MeanAbsoluteError(name="mae")
        self._mse_metric = tf.keras.metrics.MeanSquaredError(name="mse")

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)

    @property
    def metrics(self):
        return [self._mae_metric, self._mse_metric]

    def compile(self, optimizer, loss, **kwargs):
        super(AdvancedCustomModel, self).compile(**kwargs)
        self.optimizer = optimizer
        self.compiled_loss = loss

    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)
            loss = self.compiled_loss(y, y_pred)
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        self._mae_metric.update_state(y, y_pred)
        self._mse_metric.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        x, y = data
        y_pred = self(x, training=False)
        loss = self.compiled_loss(y, y_pred)
        self._mae_metric.update_state(y, y_pred)
        self._mse_metric.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}

# Usage
model = AdvancedCustomModel()
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError())

# Generate some dummy data
x_train = np.random.random((100, 4))
y_train = np.random.random((100, 1))

# Batch size
batch_size = 16

# Training using train_on_batch
for epoch in range(5):
    print(f'Start of epoch {epoch}')
    for start in range(0, len(x_train), batch_size):
        end = start + batch_size
        x_batch = x_train[start:end]
        y_batch = y_train[start:end]
        model.train_on_batch(x_batch, y_batch)

    # Print metrics at the end of each epoch
    print(f'Epoch {epoch} metrics:')
    for metric in model.metrics:
        print(f'{metric.name}: {metric.result().numpy()}')
    # Reset metrics at the end of each epoch
    for metric in model.metrics:
        metric.reset_states()

**<h2>3.  Callbacks and Custom Training Loops</h2>**

Callbacks are objects that can perform actions at various stages of training. Custom training loops provide more control over the training process.

**Example 1: Simple Custom Callback**

**Objective:** Print a message at the start and end of each epoch

In [ ]:
class SimpleCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"Starting epoch {epoch}")

    def on_epoch_end(self, epoch, logs=None):
        print(f"End of epoch {epoch}")

# Usage
model.fit(x_train, y_train, epochs=5, callbacks=[SimpleCallback()])

**Example 2: Custom Callback for Learning Rate Adjustment**

**Objective:** Adjust the learning rate dynamically

In [ ]:
class LearningRateSchedulerCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        new_lr = 0.01 * (0.1 ** (epoch // 10))
        tf.keras.backend.set_value(self.model.optimizer.lr, new_lr)
        print(f"Epoch {epoch}: Learning rate is {new_lr}")

# Usage
model.fit(x_train, y_train, epochs=20, callbacks=[LearningRateSchedulerCallback()])

**Example 3: Custom Training Loop with GradientTape**

**Objective:** Full control over training loop

In [ ]:
import tensorflow as tf
import numpy as np

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Generate some dummy data
x_train = np.random.random((100, 2))
y_train = np.random.random((100, 1))

# Define the optimizer, loss function, and metrics
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.MeanSquaredError()
train_acc_metric = tf.keras.metrics.MeanAbsoluteError()

# Define the training step function
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        y_pred = model(x, training=True)
        loss_value = loss_fn(y, y_pred)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, y_pred)
    return loss_value

# Convert the data to tf.data.Dataset and batch it
batch_size = 16
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)

# Training loop
epochs = 5
for epoch in range(epochs):
    print(f"Start of epoch {epoch}")
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        loss_value = train_step(x_batch_train, y_batch_train)
        if step % 10 == 0:
            print(f"Training loss (for one batch) at step {step}: {loss_value:.4f}")
    train_acc = train_acc_metric.result()
    print(f"Training accuracy over epoch: {train_acc:.4f}")
      .reset_states()


**<h2>4. Model Saving and Loading</h2>**

Saving and loading models in TensorFlow involves serializing the model architecture and its weights to disk and restoring them later.

**Example 1:** Save model that doesn't contain custom layer

In [ ]:
import tensorflow as tf
import numpy as np

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Generate some dummy data
x_train = np.random.random((100, 2))
y_train = np.random.random((100, 1))

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(x_train, y_train, epochs=5)

In [ ]:
# Save the model
model.save('simple_model.h5')

# Load the model
loaded_model = tf.keras.models.load_model('simple_model.h5')
loaded_model.summary()

**Example 2:** Save model that contain custom layer

In [ ]:
import tensorflow as tf
import numpy as np
import keras

@keras.saving.register_keras_serializable()
class CustomDenseLayer(tf.keras.layers.Layer):
    def __init__(self, units=32, activation=None, **kwargs):
        super(CustomDenseLayer, self).__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        self.kernel = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer='glorot_uniform',
            trainable=True
        )
        self.bias = self.add_weight(
            shape=(self.units,),
            initializer='zeros',
            trainable=True
        )
        super(CustomDenseLayer, self).build(input_shape)

    def call(self, inputs):
        output = tf.matmul(inputs, self.kernel) + self.bias
        if self.activation:
            output = self.activation(output)
        return output

    def get_config(self):
        config = super(CustomDenseLayer, self).get_config()
        config.update({
            'units': self.units,
            'activation': tf.keras.activations.serialize(self.activation),
        })
        return config

@keras.saving.register_keras_serializable()
class ModelWithCustomLayer(tf.keras.Model):
    def __init__(self, **kwargs):
        super(ModelWithCustomLayer, self).__init__(**kwargs)
        self.dense1 = tf.keras.layers.Dense(4, activation='relu')
        self.custom_layer = CustomDenseLayer(units=8, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.custom_layer(x)
        return self.dense2(x)

# Create and compile the model
model = ModelWithCustomLayer()
model.compile(optimizer='adam', loss='mse')

# Generate some dummy data
x_train = np.random.random((100, 4))
y_train = np.random.random((100, 1))

# Train the model
model.fit(x_train, y_train, epochs=5)

In [50]:
# Save the model
model.save('model_with_custom_layer.keras')

# Load the model
loaded_model = tf.keras.models.load_model('model_with_custom_layer.keras', custom_objects={'CustomDenseLayer': CustomDenseLayer})